In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt5

In [3]:
import sys

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.PosarMCParameters import *

In [9]:
scalingFactor = 2 / 65535
offset = -1

### Read first set of data

In [31]:
data_date_c = "2018_03_29_16_39_48"
data_dir = "/home/pleroy/DATA/PoSAR-MC_v2/2018_03_28-29/" + data_date_c

params_filename = data_dir + "/" + data_date_c + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )
buffer_c = 0
filename = data_dir + '/record' + str(buffer_c) + '.bin'
fd = open(filename,'rb')
record = np.fromfile(fd, dtype = np.uint16, count = params.rampsPerFile * params.samplesPerRamp * 2)
record_0 = record[::2].reshape(int(params.rampsPerFile), params.samplesPerRamp)
record_1 = record[1::2].reshape(int(params.rampsPerFile), params.samplesPerRamp)

raw_c = record_0.reshape(params.rampsPerFile, params.samplesPerRamp)
scaled_c = raw_c * scalingFactor + offset

### Read parameters.xml

In [14]:
data_date = "2018_04_06_14_06_48"
data_dir = "/home/pleroy/DATA/PoSAR-v2_PIMA_TEST-1/" + data_date

In [15]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )

In [16]:
samplesPerFile = params.rampsPerFile * params.samplesPerRamp

In [17]:
print(params.rampsPerFile * params.samplesPerRamp * 4)

36000000


### Process timeStamps.data

In [18]:
timeStampsFile = data_dir + "/" + data_date + "_timeStamps.data"
blockNumber, timeStamp = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

In [19]:
plt.figure()
plt.suptitle(data_date)

plt.subplot(211)
plt.plot( blockNumber, timeStamp, '.', label="timeStamps" )
plt.grid()
plt.legend()

plt.subplot(212)
plt.plot( np.diff(timeStamp[1:]/1000), '.', label="diff timeStamps" )
plt.grid()
plt.legend()

### Read second set of data

In [20]:
firstFileIsFull = 1

In [21]:
buffer_a = 0
filename_a = data_dir + '/record' + str(buffer_a) + '.bin'
fd_a = open(filename_a,'rb')
if firstFileIsFull == 1:
    rampsToRead = params.rampsPerFile
else:
    rampsToRead = int(params.rampsPerFile / 2)
record_a = np.fromfile(fd_a, dtype = np.uint16, count = rampsToRead * params.samplesPerRamp * 2)
record_a_0 = record_a[::2].reshape(rampsToRead, params.samplesPerRamp)
record_a_1 = record_a[1::2].reshape(rampsToRead, params.samplesPerRamp)

buffer_b = 442
filename_b = data_dir + '/record' + str(buffer_b) + '.bin'
fd_b = open(filename_b,'rb')
record_b = np.fromfile(fd_b, dtype = np.uint16, count = params.rampsPerFile * params.samplesPerRamp * 2)
record_b_0 = record_b[::2].reshape(params.rampsPerFile, params.samplesPerRamp)
record_b_1 = record_b[1::2].reshape(params.rampsPerFile, params.samplesPerRamp)

In [22]:
raw_a = record_a_0.reshape(rampsToRead, params.samplesPerRamp)
raw_b = record_b_0.reshape(params.rampsPerFile, params.samplesPerRamp)

In [24]:
scaled_a = raw_a * scalingFactor + offset
scaled_b = raw_b * scalingFactor + offset

### Plot data

In [25]:
font = { 'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)

In [32]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(raw_a[0,:], color='b', label = str(buffer_a) + ' ' + data_date)
ax1.plot(raw_c[0,:], color='g', label = str(buffer_c) + ' ' + data_date_c)
ax1.set_ylabel('raw')
ax1.legend()

ax2.plot(scaled_a[0,:], color='b', label = str(buffer_a) + ' ' + data_date)
ax2.plot(scaled_c[0,:], color='g', label = str(buffer_c) + ' ' + data_date_c)
ax2.set_ylabel('scaled [V]')
    
ax1.grid()

### Plot spectrum

In [35]:
nbSamplesPerHalfRamp = int(params.samplesPerRamp/2)

In [36]:
firstRamp = raw_a[0, 0:nbSamplesPerHalfRamp]
firstRampScaled = firstRamp * scalingFactor + offset
fft = np.fft.fft(firstRampScaled)

firstRamp_b = raw_b[0, 0:nbSamplesPerHalfRamp]
firstRampScaled_b = firstRamp_b * scalingFactor + offset
fft_b = np.fft.fft(firstRampScaled_b)

firstRamp_c = raw_c[0, 0:nbSamplesPerHalfRamp]
firstRampScaled_c = firstRamp_c * scalingFactor + offset
fft_c = np.fft.fft(firstRampScaled_c)

In [37]:
plt.figure()
plt.plot(firstRampScaled, label=data_date)
plt.plot(firstRampScaled_b, label=data_date)
plt.plot(firstRampScaled_c, label=data_date)
plt.legend()
plt.grid()

In [38]:
max_fft = 20 * np.log10( np.abs( np.max(np.abs(fft)) ) )
print("max fft = {:.2f} dB".format(max_fft))

max fft = 35.78 dB


In [39]:
plt.figure()
plt.plot(20*np.log10(np.abs(fft)), label=data_date)
plt.plot(20*np.log10(np.abs(fft_b)), label=data_date)
plt.plot(20*np.log10(np.abs(fft_c)), label=data_date_c)
plt.grid()
plt.legend()